# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [41]:
import torch
import os
from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

In [23]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

True


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [24]:
# set the path to your local lmdb directory
train_src = "s2ef"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [25]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [31]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [32]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 8,
    'lr_initial': 0.0001,
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.8,
    'patience': 3,
    'max_epochs': 80,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [33]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [34]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-09-04-08-51-28-SchNet-example
  commit: 98a06d8
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-09-04-08-51-28-SchNet-example
  print_every: 5
  results_dir: ./results/2021-09-04-08-51-28-SchNet-example
  seed: 0
  timestamp_id: 2021-09-04-08-51-28-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 1
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  factor: 0.8
  force_coefficient: 100
  lr_initial: 0.0001
  max_epochs: 1
  mode: min
  num_workers: 8
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true
  type: regres

2021-09-04 08:51:37 (WARNING): Model gradient logging to tensorboard not yet supported.


## Check the model

In [35]:
print(trainer.model)

OCPDataParallel(
  (module): SchNet(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [36]:
trainer.train()

2021-09-04 08:51:43 (INFO): forcesx_mae: 6.12e-01, forcesy_mae: 7.54e-01, forcesz_mae: 7.98e-01, forces_mae: 7.21e-01, forces_cos: -8.32e-03, forces_magnitude: 1.34e+00, energy_mae: 3.14e+01, energy_force_within_threshold: 0.00e+00, loss: 1.04e+02, lr: 1.00e-04, epoch: 1.25e-01, step: 5.00e+00
2021-09-04 08:51:43 (INFO): forcesx_mae: 4.95e-01, forcesy_mae: 5.85e-01, forcesz_mae: 6.06e-01, forces_mae: 5.62e-01, forces_cos: -1.64e-03, forces_magnitude: 9.97e-01, energy_mae: 2.38e+01, energy_force_within_threshold: 0.00e+00, loss: 8.02e+01, lr: 1.00e-04, epoch: 2.50e-01, step: 1.00e+01
2021-09-04 08:51:44 (INFO): forcesx_mae: 4.35e-01, forcesy_mae: 5.44e-01, forcesz_mae: 5.30e-01, forces_mae: 5.03e-01, forces_cos: 2.57e-02, forces_magnitude: 9.14e-01, energy_mae: 2.09e+01, energy_force_within_threshold: 0.00e+00, loss: 7.11e+01, lr: 1.00e-04, epoch: 3.75e-01, step: 1.50e+01
2021-09-04 08:51:44 (INFO): forcesx_mae: 3.70e-01, forcesy_mae: 4.50e-01, forcesz_mae: 4.22e-01, forces_mae: 4.14e-0

device 0: 100%|██████████| 79/79 [00:01<00:00, 39.87it/s]

2021-09-04 08:51:48 (INFO): forcesx_mae: 0.2778, forcesy_mae: 0.3467, forcesz_mae: 0.3606, forces_mae: 0.3284, forces_cos: 0.0278, forces_magnitude: 0.5615, energy_mae: 12.4560, energy_force_within_threshold: 0.0000, loss: 44.8795, epoch: 1.0000


2021-09-04 08:51:49 (INFO): Predicting on test.


device 0: 100%|██████████| 79/79 [00:01<00:00, 41.47it/s]

2021-09-04 08:51:51 (INFO): Writing results to ./results/2021-09-04-08-51-28-SchNet-example/s2ef_predictions.npz


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [37]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints/2021-09-04-08-51-28-SchNet-example/checkpoint.pt'

In [38]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_checkpoint(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-09-04-08-51-28-SchNet-example
  commit: 98a06d8
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-09-04-08-51-28-SchNet-example
  print_every: 10
  results_dir: ./results/2021-09-04-08-51-28-SchNet-example
  seed: 0
  timestamp_id: 2021-09-04-08-51-28-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 1
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  factor: 0.8
  force_coefficient: 100
  lr_initial: 0.0001
  max_epochs: 1
  mode: min
  num_workers: 8
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true
  type: regre

2021-09-04 08:51:51 (WARNING): Model gradient logging to tensorboard not yet supported.


2021-09-04 08:51:51 (INFO): Loading checkpoint from: ./checkpoints/2021-09-04-08-51-28-SchNet-example/checkpoint.pt


## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [39]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

2021-09-04 08:51:51 (INFO): Predicting on test.


device 0: 100%|██████████| 79/79 [00:01<00:00, 44.68it/s]

2021-09-04 08:51:53 (INFO): Writing results to ./results/2021-09-04-08-51-28-SchNet-example/s2ef_s2ef_results.npz


In [40]:
energies = predictions["energy"]
forces = predictions["forces"]